In [1]:
## STEP 1: Set up the workspace.

import json

#GETTING MY ID:
id = open('my_id.json', 'r')
mi = json.load(id)
my_id = mi["my_id"]

from azureml.core import Workspace

ws = Workspace.create(name="ModifiedDate_Prediction",
                      subscription_id = my_id,
                      resource_group = "GRUPOSQL_3",
                      location = "centralindia")

from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="model.pkl",
                                  model_name=mname,
                                  workspace=ws)

UserErrorException: UserErrorException:
	Message: You are currently logged-in to c65a3ea6-0f7c-400b-8934-5a6dc1705645 tenant. You don't have access to 17ac9dfc-c41b-4b57-a311-eaec57365613 subscription, please check if it is in this tenant. All the subscriptions that you have access to in this tenant are = 
 [SubscriptionInfo(subscription_name='Visual Studio Enterprise Subscription Cad', subscription_id='90eaa2d9-15dc-4bc3-9c4f-2c580923e179'), SubscriptionInfo(subscription_name='Azure subscription 1', subscription_id='aa0fd98f-926d-4a64-ae2a-d4fc69292cd9')]. 
 Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "You are currently logged-in to c65a3ea6-0f7c-400b-8934-5a6dc1705645 tenant. You don't have access to 17ac9dfc-c41b-4b57-a311-eaec57365613 subscription, please check if it is in this tenant. All the subscriptions that you have access t

In [ ]:
################ score.py ###################
import json

scorepy = f"""
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
  global model
  model_path = Model.get_model_path('{mname}')
  model = joblib.load(model_path)



def run(raw_data):
  try: ## Try la predicción.
    data = json.loads(raw_data)['data'][0]
    data = pd.DataFrame(data)
    
    #Realizar prediccion
    result = model.predict(data).tolist()

    return json.dumps(result_finals)
  except Exception as e:
    return json.dumps(str(e))
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

In [ ]:
## STEP 2: Inference configuration: the blueprints for the German carpenter about how to build your furniture.

from azureml.core.environment import Environment
virtual_env = Environment("env-4-housing")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','scikit-learn'])


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name='banruptcy-prediction-service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

In [ ]:
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

In [ ]:
print(scoring_uri)